In [25]:
import glob
import json
from tqdm import tqdm
import os
import json
import pandas as pd
import re
import polars as pl


# Download and extract data

Download and extract a dump from Wikipedia
```
wget -O simplewiki-20250301-pages-articles-multistream.xml.bz2 https://dumps.wikimedia.org/simplewiki/20250301/simplewiki-20250301-pages-articles-multistream.xml.bz2
```

```
bzip2 -d simplewiki-20250301-pages-articles-multistream.xml.bz2 
```

Use WikiExtractor to convert XML into a number of JSON files
```
python -m wikiextractor.WikiExtractor simplewiki-20250301-pages-articles-multistream.xml --json --output extracted
```

In [26]:
files = glob.glob('../extracted/*/*')
files

['../extracted/AA/wiki_28',
 '../extracted/AA/wiki_70',
 '../extracted/AA/wiki_72',
 '../extracted/AA/wiki_95',
 '../extracted/AA/wiki_09',
 '../extracted/AA/wiki_81',
 '../extracted/AA/wiki_99',
 '../extracted/AA/wiki_17',
 '../extracted/AA/wiki_03',
 '../extracted/AA/wiki_20',
 '../extracted/AA/wiki_00',
 '../extracted/AA/wiki_05',
 '../extracted/AA/wiki_47',
 '../extracted/AA/wiki_21',
 '../extracted/AA/wiki_42',
 '../extracted/AA/wiki_11',
 '../extracted/AA/wiki_76',
 '../extracted/AA/wiki_57',
 '../extracted/AA/wiki_51',
 '../extracted/AA/wiki_02',
 '../extracted/AA/wiki_88',
 '../extracted/AA/wiki_53',
 '../extracted/AA/wiki_46',
 '../extracted/AA/wiki_39',
 '../extracted/AA/wiki_49',
 '../extracted/AA/wiki_12',
 '../extracted/AA/wiki_61',
 '../extracted/AA/wiki_59',
 '../extracted/AA/wiki_29',
 '../extracted/AA/wiki_16',
 '../extracted/AA/wiki_14',
 '../extracted/AA/wiki_54',
 '../extracted/AA/wiki_67',
 '../extracted/AA/wiki_50',
 '../extracted/AA/wiki_66',
 '../extracted/AA/wi

In [14]:
import polars as pl
x= pl.scan_parquet('./000_00000.parquet')
#x.head().collect()
x = x.filter((pl.col('token_count')>100)&(pl.col('score')>2.5)&(pl.col('language')=='en'))
x = x.collect()

In [15]:
print(x.filter(x['language_score']<0.7)['text'][0])

In the translated text above, these terms all get the articles, however, in the test, they all don't.
i.e.:la astronomía, y en las matemáticas. También, trabajaron en la agricultura y el comercio"
in the test, it corrected me, so I'm a bit confused:
Could you please let me know when you can/cannot use the article "le,la,las" etc in front of these:
The test examples:
Kwizbot Los aztecas destacaron en astronomía,
You Los aztecas destacan en la astronomía,
You could also say: Los aztecas destacaron en la astronomía,
Kwizbot También, trabajaron en la agricultura
You También, trabajaron en agricultura
[note: here I dropped the article and it didn't correct me]
Well done! and in mathematics.
Your answer matched mine: y en matemáticas.
Thank you for your help and have a great day!
Freeform Writing Exercise A2
Four years later, and still no answer to this question. I had the same questions. It appears to me using various online resources, grammar correcting sites, and ChatGPT that the use of a

In [16]:
print(x.filter(x['score']<2.7)['text'][10])

Welcome to our comprehensive guide on Argon2 vs PBKDF2, two of the most popular password hashing algorithms in the world of cybersecurity. In this article, we’ll explore the key differences between Argon2 and PBKDF2, examining their security features, performance, and best use cases. Whether you are a developer looking to enhance your application’s security or simply curious about the best practices for protecting your sensitive data, you’ll find valuable insights here. Join us as we break down the strengths and weaknesses of each hashing method, helping you make an informed decision for your password management needs.
What is Argon2
Argon2 is a modern cryptographic hash function specifically designed for password hashing and key derivation. Developed in 2015 and the winner of the Password Hashing Competition (PHC), Argon2 focuses on security against both brute-force attacks and specialized hardware attacks. It comes in two variants: Argon2d, which is resistant to GPU cracking attacks,

In [21]:
import pandas as pd
import google.generativeai as genai
import os
import time
import json
from dotenv import load_dotenv
from tqdm import tqdm  # Import tqdm for progress bar

load_dotenv()
API_KEY = os.getenv("MY_API_KEY")

if not API_KEY:
    raise ValueError("Google API Key not found. Please set the GOOGLE_API_KEY environment variable or define it in the script.")

genai.configure(api_key=API_KEY)


MODEL_NAME = "gemini-1.5-flash-8b"

ALLOWED_TOPICS = ["Art", "History", "Science", "General knowledge"]
ALLOWED_COMPLEXITY = ["elementary", "middle school", "high school", "university degree", "doctorate"]

def classify_text_with_gemini(text_to_classify):
    """
    Uses Gemini to classify text into topic and complexity.

    Args:
        text_to_classify (str): The text content to classify.

    Returns:
        tuple: (topic, complexity) or (None, None) if an error occurs.
    """
    # Ensure text is not empty or just whitespace
    if not text_to_classify or text_to_classify.isspace():
        print("Warning: Skipping empty or whitespace-only text.")
        return None, None

    # Limit text length to avoid exceeding model limits (adjust as needed)
    max_length = 15000 # Example limit, check model's token limits if necessary
    if len(text_to_classify) > max_length:
        text_to_classify = text_to_classify[:max_length] + "..." # Truncate


    prompt = f"""
Analyze the following text and determine its primary topic and the educational level typically required to understand it.

Text:
"{text_to_classify}"

Instructions:
1. Choose the *single best* topic from this list: {ALLOWED_TOPICS}
2. Choose the *single most appropriate* complexity level from this list: {ALLOWED_COMPLEXITY}
3. Provide your answer ONLY in the following JSON format:
   {{"topic": "SELECTED_TOPIC", "complexity": "SELECTED_COMPLEXITY"}}

Example Response:
{{"topic": "Science", "complexity": "high school"}}

Output ONLY the JSON object.
"""

    try:
        model = genai.GenerativeModel(MODEL_NAME)
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                # candidate_count=1, # Ensure only one response candidate
                # max_output_tokens=50, # Limit output size
                temperature=0.1 # Lower temperature for more deterministic classification
            )
            )

        raw_response_text = response.text.strip()
        if raw_response_text.startswith("```json"):
            raw_response_text = raw_response_text[7:] # Remove ```json
        if raw_response_text.endswith("```"):
            raw_response_text = raw_response_text[:-3] # Remove ```
        raw_response_text = raw_response_text.strip() # Clean whitespace again

        # Parse the JSON response
        result = json.loads(raw_response_text)
        topic = result.get("topic")
        complexity = result.get("complexity")

        # Validate response against allowed categories
        if topic not in ALLOWED_TOPICS:
            print(f"Warning: Received invalid topic '{topic}'. Setting to None.")
            topic = None
        if complexity not in ALLOWED_COMPLEXITY:
            print(f"Warning: Received invalid complexity '{complexity}'. Setting to None.")
            complexity = None

        return topic, complexity

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response: {raw_response_text}")
        return "Error: JSON Decode", "Error: JSON Decode"
    except ValueError as ve: # Handle potential errors from model generation (e.g., blocked content)
         print(f"Error: Gemini API returned ValueError (potentially blocked content or invalid response structure). Raw response: {response.text if 'response' in locals() else 'N/A'}. Details: {ve}")
         return "Error: API Value", "Error: API Value"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Error: API Call", "Error: API Call"

In [19]:
df = pd.DataFrame(columns=['text'])

df['text'] = x['text'][0:100].to_list()
df

,text
0,This project is solving the Asteroid Watchers ...
1,Life of a Sand Grain\nTHE LIFE OF A SAND GRAIN...
2,"The internet of things, a system of interrelat..."
3,An archive photo of an Egyptian mummy - Reuter...
4,The faith of the Christ-God is a living parado...
...,...
95,Extreme weather events are generally associate...
96,"In diary studies, people provide frequent repo..."
97,A 10 minutes read to dive into the REFLOW proj...
98,More than lullabies and swaying. Kindermusik i...


In [24]:
classify_text_with_gemini('Questo è un racconto per bambini')

('General knowledge', 'elementary')

In [31]:
import concurrent.futures
MAX_WORKERS = 8
results = [None] * len(df) # Pre-allocate list to store results in order

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:

    future_to_index = {
        executor.submit(classify_text_with_gemini, text): i
        for i, text in enumerate(df['text'])
    }

    # Process futures as they complete, using tqdm for progress
    for future in tqdm(concurrent.futures.as_completed(future_to_index), total=len(df), desc="Classifying Text"):
        index = future_to_index[future]
        try:
            # Get the result tuple (topic, complexity)
            result_tuple = future.result()
            results[index] = result_tuple
        except Exception as exc:
            print(f'Row {index} generated an exception: {exc}')
            # Store error indication in results for the specific row
            results[index] = ("Error: Future Exception", "Error: Future Exception")


Classifying Text: 100%|████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.61it/s]


In [32]:
x['token_count'].sum()

412145408

In [23]:
data_rows = []

for file_path in tqdm(files):
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            try:
                record = json.loads(line)
                title = record.get("title", "")
                text = record.get("text", "")
                #tfidf_topics = detect_topics_by_frequency(text)
                
                #topic_label = ", ".join(tfidf_topics)
                
                data_rows.append({
                    "filename": file_path,
                    "title": title,
                    "text": text,
                    "number_of_characters": len(text),
                    "number_of_words": len(text.split()),
                    "topic": 'N/A',
                    "text_quality": 0,
                    
                })
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {file_path}: {e}")

df = pd.DataFrame(data_rows)
print(df)

100%|██████████████████████████████████████████████████████████████████████████████| 233/233 [00:04<00:00, 54.21it/s]


                      filename                             title  \
0       ./extracted/AB/wiki_63                      Michel Debré   
1       ./extracted/AB/wiki_63                    Michel Barnier   
2       ./extracted/AB/wiki_63                 Hervé de Charette   
3       ./extracted/AB/wiki_63            Sweet and Sour (movie)   
4       ./extracted/AB/wiki_63                       Dafydd Iwan   
...                        ...                               ...   
367504  ./extracted/AC/wiki_29                   Jonathan Majors   
367505  ./extracted/AC/wiki_29                     Bajwa (caste)   
367506  ./extracted/AC/wiki_29             2025 Tibet earthquake   
367507  ./extracted/AC/wiki_29  Internment of Japanese Canadians   
367508  ./extracted/AC/wiki_29                       Leap second   

                                                     text  \
0       Michel Jean-Pierre Debré (; 15 January 1912 – ...   
1       Michel Bernard Barnier (born 9 January 1951) i...   


In [26]:
df['number_of_words'].sum()/0.7

45825104.28571429

In [5]:
df.to_parquet('simple_english.parquet')

In [27]:
df.iloc[0]['text']

'Michel Jean-Pierre Debré (; 15 January 1912 – 2 August 1996) was a French politician. He was the first Prime Minister of the French Fifth Republic. He has been called the "father" of the current Constitution of France. He worked with President Charles de Gaulle from 1959 to 1962. '

In [66]:
from transformers import PreTrainedTokenizerFast

enc = PreTrainedTokenizerFast.from_pretrained('./dev/data/simple_english.parquet/custom_tokenizer')

with open('./dev/data/simple_english.parquet/simple_english.parquet_custom_train_000041.bin', "rb") as f:
    # first read the header, which is 256 int32 integers (4 bytes each)
    header = np.frombuffer(f.read(256*4), dtype=np.int32)
    ntok = header[2] # number of tokens (claimed)
    # the rest of it are tokens, stored as uint16
    tokens = np.frombuffer(f.read(), dtype=np.uint32)
    
    
decoded_text = enc.decode(tokens[100:200]).replace(" ", "").replace("Ġ", " ").replace(" ##", "").replace('Ċ','\n')

print(decoded_text)

 originated from English speaking countries. It finds seldom use in other metricated Commonwealth Nations.</s></s></s> Shohreh Solati (, born Fatemeh Solati on January 4, 1959, in Tehran) is an Iranian singer. She is among the most active and prolific Iranian female singers. Since the Iranian Revolution, she has continued her music career outside the country.
She is divorced with a daughter and lives in Los Angeles, California.</s></s>
